# Evaluation of the models

### Task breakdown:

#### 1. Load Benchmark data for each query.

#### 2. Load Ranked Documents for each query.

#### 3. Evaluate the data

#### 4. Compare the models from the results obtained


### Testing code for evaluating model for 1 query with 1 model's result for that query

For Query R101 with benchmark 'Dataset101' and result with BM25 Model for Query 1 

In [44]:
query_benchmark_101_file_path = 'EvaluationBenchmark/Dataset101.txt'
bm25model_r101_result ='My Code/Outputs-Task1-New/BM25_R101Ranking.dat' 

In [45]:
if __name__ == "__main__":

    import sys
    import os
    #import coll
    #import df


    # task 2 evaluation
    # get the benchmark
    benFile = open(query_benchmark_101_file_path)
    #benFile = open('Training_benchmark.txt')
    file_ = benFile.readlines()
    ben={}
    for line in file_:
        line = line.strip()
        lineList = line.split()
        ben[lineList[1]]=float(lineList[2])
    benFile.close()
    
    # Print the loaded benchmark data
    print("Loaded Benchmark Data:")
    print(ben)
    
    # number documents 
    rank1 = {}
    i = 1
    for line in open(bm25model_r101_result):
        line = line.strip()
        lineList = line.split()
        # Strip '.xml' extension from document IDs
        doc_id = lineList[0].replace('.xml', '')
        rank1[str(i)] = doc_id
        i += 1
        
    #Print the loaded ranked documents
    print("Loaded Ranked Documents:")
    print(rank1)

    
    # Evaluation
    print("For task 2:")
    ri = 0
    map1 = 0.0
    R = len([id for (id, v) in ben.items() if v > 0])
    if R == 0:
        print("No relevant documents in the benchmark.")
    else:
        for (n, doc_id) in sorted(rank1.items(), key=lambda x: int(x[0])):
            print(f"Evaluating Document ID: {doc_id} at Rank: {n}")
            if doc_id in ben:
                if ben[doc_id] > 0:
                    ri += 1
                    pi = float(ri) / float(int(n))
                    recall = float(ri) / float(R)
                    map1 += pi
                    print(f"At position {int(n)}, precision= {pi}, recall= {recall}")
                else:
                    print(f"Document ID: {doc_id} is found in benchmark but not relevant.")
            else:
                # Instead of printing not found in benchmark for each document, count the not found documents
                print(f"Document ID: {doc_id} is not found in benchmark.")
        if ri > 0:
            map1 = map1 / float(ri)
        else:
            map1 = 0
        print("---The average precision = " + str(map1))

        if ri == 0:
            print("No relevant documents found in the ranked results.")


Loaded Benchmark Data:
{'6146': 0.0, '18586': 0.0, '22170': 0.0, '22513': 0.0, '26642': 0.0, '26847': 0.0, '27577': 0.0, '30647': 0.0, '39496': 1.0, '46547': 1.0, '46974': 1.0, '61329': 0.0, '61780': 0.0, '62325': 1.0, '63261': 1.0, '77909': 0.0, '80425': 0.0, '80950': 0.0, '81463': 0.0, '82330': 1.0, '82454': 1.0, '82912': 0.0, '83167': 0.0}
Loaded Ranked Documents:
{'1': '61780', '2': '63261', '3': '46974', '4': '46547', '5': '22170', '6': '62325', '7': '6146', '8': '22513', '9': '61329', '10': '39496', '11': '77909', '12': '82330', '13': '18586', '14': '80950', '15': '83167', '16': '82454', '17': '27577', '18': '26847', '19': '30647', '20': '80425', '21': '26642', '22': '81463', '23': '82912'}
For task 2:
Evaluating Document ID: 61780 at Rank: 1
Document ID: 61780 is found in benchmark but not relevant.
Evaluating Document ID: 63261 at Rank: 2
At position 2, precision= 0.5, recall= 0.14285714285714285
Evaluating Document ID: 46974 at Rank: 3
At position 3, precision= 0.6666666666666

### Now Testing code for evaluating model for all the queries benchmark with BM25 model's results for those querries.

For queries 101 to 150 with benchmarks 101...150 for BM25

In [43]:
import os

def load_benchmark(benchmark_file):
    with open(benchmark_file) as benFile:
        file_ = benFile.readlines()
    ben = {}
    for line in file_:
        line = line.strip()
        lineList = line.split()
        ben[lineList[1]] = float(lineList[2])
    return ben

def load_ranked_results(results_file):
    rank = {}
    with open(results_file) as f:
        i = 1
        for line in f:
            line = line.strip()
            lineList = line.split()
            # Strip '.xml' extension from document IDs
            doc_id = lineList[0].replace('.xml', '')
            rank[str(i)] = doc_id
            i += 1
    return rank


def evaluate_query(ben, rank1):
    ri = 0
    map1 = 0.0
    R = len([id for (id, v) in ben.items() if v > 0])
    if R == 0:
        return 0, 0, 0  # No relevant documents in the benchmark
    for (n, doc_id) in sorted(rank1.items(), key=lambda x: int(x[0])):
        if doc_id in ben and ben[doc_id] > 0:
            ri += 1
            pi = float(ri) / float(int(n))
            map1 += pi
    if ri > 0:
        map1 = map1 / float(ri)
    else:
        map1 = 0
    recall = float(ri) / float(R) if R > 0 else 0
    return ri, recall, map1

def print_performance_table(performance_data):
    print("\nPerformance Table:")
    print(f"{'Topic':<10} | {'BM25':<10}")
    print("-" * 22)
    for topic, map_value in performance_data:
        print(f"{topic:<10} | {map_value:<10}")

def main():
    query_range = range(101, 151)  # Adjust range as needed
    benchmark_folder = "EvaluationBenchmark"
    results_folder = "My Code/Outputs-Task1-New"
    
    total_relevant_items = 0
    total_recall = 0
    total_map = 0
    total_queries = 0

    performance_data = []

    for query_id in query_range:
        benchmark_file = os.path.join(benchmark_folder, f"Dataset{query_id}.txt")
        results_file = os.path.join(results_folder, f"BM25_R{query_id}Ranking.dat")
        
        if os.path.exists(benchmark_file) and os.path.exists(results_file):
            ben = load_benchmark(benchmark_file)
            rank1 = load_ranked_results(results_file)
            ri, recall, map1 = evaluate_query(ben, rank1)
            total_relevant_items += ri
            total_recall += recall
            total_map += map1
            total_queries += 1
            print(f"For Query {query_id}: relevant documents (ri)={ri}, recall={recall}, MAP={map1}")
            performance_data.append((f"R{query_id}", map1))
        else:
            print(f"Files for query {query_id} do not exist.")
    
    if total_queries > 0:
        avg_recall = total_recall / total_queries
        avg_map = total_map / total_queries
    else:
        avg_recall = 0
        avg_map = 0

    print(f"\nOverall results for {total_queries} queries:")
    print(f"Total relevant items found: {total_relevant_items}")
    print(f"Average recall: {avg_recall}")
    print(f"Average MAP: {avg_map}")

    # Call the performance table function
    print_performance_table(performance_data)

if __name__ == "__main__":
    main()


For Query 101: relevant documents (ri)=7, recall=1.0, MAP=0.5744047619047619
For Query 102: relevant documents (ri)=135, recall=1.0, MAP=0.6363322454881951
For Query 103: relevant documents (ri)=14, recall=1.0, MAP=0.22607885048707604
For Query 104: relevant documents (ri)=120, recall=1.0, MAP=0.4822634740278421
For Query 105: relevant documents (ri)=16, recall=1.0, MAP=0.4478223326042083
For Query 106: relevant documents (ri)=4, recall=1.0, MAP=0.43599791013584116
For Query 107: relevant documents (ri)=3, recall=1.0, MAP=0.53125
For Query 108: relevant documents (ri)=3, recall=1.0, MAP=0.08409961685823757
For Query 109: relevant documents (ri)=20, recall=1.0, MAP=0.3985829218274427
For Query 110: relevant documents (ri)=5, recall=1.0, MAP=0.26757964257964256
For Query 111: relevant documents (ri)=3, recall=1.0, MAP=0.11858076563958915
For Query 112: relevant documents (ri)=6, recall=1.0, MAP=0.4932178932178932
For Query 113: relevant documents (ri)=12, recall=1.0, MAP=0.34900356664664

### Now Testing code for evaluating model for all the queries benchmark with JM_LM model's results for those querries.

In [39]:
import os
import pandas as pd

def load_benchmark(benchmark_file):
    with open(benchmark_file) as benFile:
        file_ = benFile.readlines()
    ben = {}
    for line in file_:
        line = line.strip()
        lineList = line.split()
        ben[lineList[1]] = float(lineList[2])
    return ben

def load_ranked_results(results_file):
    rank = {}
    with open(results_file) as f:
        i = 1
        for line in f:
            line = line.strip()
            lineList = line.split()
            # Strip '.xml' extension from document IDs
            doc_id = lineList[0].replace('.xml', '')
            rank[str(i)] = doc_id
            i += 1
    return rank


def evaluate_query(ben, rank1):
    ri = 0
    map1 = 0.0
    R = len([id for (id, v) in ben.items() if v > 0])
    if R == 0:
        return 0, 0, 0  # No relevant documents in the benchmark
    for (n, doc_id) in sorted(rank1.items(), key=lambda x: int(x[0])):
        if doc_id in ben and ben[doc_id] > 0:
            ri += 1
            pi = float(ri) / float(int(n))
            map1 += pi
    if ri > 0:
        map1 = map1 / float(ri)
    else:
        map1 = 0
    recall = float(ri) / float(R) if R > 0 else 0
    return ri, recall, map1

def create_performance_table(performance_data, models):
    df_data = {'Topic': []}
    for model in models:
        df_data[model] = []

    for topic, scores in performance_data.items():
        df_data['Topic'].append(topic)
        for model in models:
            df_data[model].append(scores.get(model, "N/A"))

    df = pd.DataFrame(df_data)
    return df

def main():
    query_range = range(101, 151)  # Adjust range as needed
    benchmark_folder = "EvaluationBenchmark"
    models = {
        "BM25": "My Code/RankingOutputs/BM25_R{query_id}Ranking.dat",
        "JM_LM": "My Code/RankingOutputs/JM_R{query_id}Ranking.dat"
    }
    
    performance_data = {f"R{query_id}": {} for query_id in query_range}

    for model_name, results_pattern in models.items():
        for query_id in query_range:
            benchmark_file = os.path.join(benchmark_folder, f"Dataset{query_id}.txt")
            results_file = results_pattern.format(query_id=query_id)
            
            if os.path.exists(benchmark_file) and os.path.exists(results_file):
                ben = load_benchmark(benchmark_file)
                rank1 = load_ranked_results(results_file)
                ri, recall, map1 = evaluate_query(ben, rank1)
                #print(f"For Query {query_id} ({model_name}): relevant documents (ri)={ri}, recall={recall}, MAP={map1}")
                performance_data[f"R{query_id}"][model_name] = map1
            else:
                #print(f"Files for query {query_id} ({model_name}) do not exist.")
                performance_data[f"R{query_id}"][model_name] = "N/A"

    # Create DataFrame
    performance_df = create_performance_table(performance_data, models.keys())
    print(performance_df)

if __name__ == "__main__":
    main()


   Topic      BM25     JM_LM
0   R101  0.000000  0.000000
1   R102  0.000000  0.000000
2   R103  0.000000  0.000000
3   R104  0.000000  0.000000
4   R105  0.000000  0.000000
5   R106  0.000000  0.000000
6   R107  0.000000  0.000000
7   R108  0.000000  0.000000
8   R109  0.000000  0.000000
9   R110  0.000000  0.000000
10  R111  0.000000  0.000000
11  R112  0.000000  0.000000
12  R113  0.000000  0.000000
13  R114  0.000000  0.000000
14  R115  0.000000  0.000000
15  R116  0.000000  0.000000
16  R117  0.000000  0.000000
17  R118  0.000000  0.000000
18  R119  0.000000  0.000000
19  R120  0.000000  0.000000
20  R121  0.000000  0.000000
21  R122  0.000000  0.000000
22  R123  0.000000  0.000000
23  R124  0.000000  0.000000
24  R125  0.000000  0.000000
25  R126  0.000000  0.000000
26  R127  0.000000  0.000000
27  R128  0.000000  0.000000
28  R129  0.000000  0.000000
29  R130  0.000000  0.000000
30  R131  0.000000  0.000000
31  R132  0.000000  0.000000
32  R133  0.000000  0.000000
33  R134  0.00